# Homework Week1

## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19


In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [2]:
df01 = pd.read_parquet('./Data/yellow_tripdata_2023-01.parquet')

In [3]:
len(df01.columns)

19

### Ans: 19

## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

* 32.59
* 42.59
* 52.59
* 62.59

In [4]:
df01.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

In [5]:
df01['duration'] = (df01['tpep_dropoff_datetime'] - df01['tpep_pickup_datetime']).dt.total_seconds()/60

In [6]:
std = np.std(df01['duration'])

In [7]:
std

np.float64(42.59434429744777)

### Ans: 42.59

## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [8]:
train_set = df01[(df01['duration']>=1)&(df01['duration']<= 60)]

In [9]:
len(train_set)/len(df01)

0.9812202822125979

### Ans: 98%

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [15]:
train_set['PULocationID'] = train_set['PULocationID'].astype(str)
train_set['DOLocationID'] = train_set['DOLocationID'].astype(str)

/tmp/ipykernel_7393/2896583935.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['PULocationID'] = train_set['PULocationID'].astype(str)
/tmp/ipykernel_7393/2896583935.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_set['DOLocationID'] = train_set['DOLocationID'].astype(str)


In [23]:
train_set.dtypes

VendorID                          int64
tpep_pickup_datetime     datetime64[us]
tpep_dropoff_datetime    datetime64[us]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                     object
DOLocationID                     object
payment_type                      int64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
airport_fee                     float64
duration                        float64
dtype: object

In [24]:
X_train = train_set[['PULocationID', 'DOLocationID']].to_dict(orient = 'records')

In [25]:
dv = DictVectorizer()
X_train = dv.fit_transform(X_train)

In [18]:
X_train.shape

(3009173, 515)

### Ans: 515

## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

* Train a plain linear regression model with default parameters, where duration is the response variable
* Calculate the RMSE of the model on the training data
What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [26]:
y_train = train_set['duration'].values

In [27]:
prediction_model = LinearRegression()
prediction_model.fit(X_train, y_train)
y_pred = prediction_model.predict(X_train)

In [28]:
print("RMSE: ", root_mean_squared_error(y_train, y_pred))

RMSE:  7.649261934850555


### Ans: 7.64

## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81


In [30]:
df02 = pd.read_parquet('./Data/yellow_tripdata_2023-02.parquet')

In [31]:
df02['duration'] = (df02['tpep_dropoff_datetime']-df02['tpep_pickup_datetime']).dt.total_seconds()/60
df02 = df02[(df02['duration']>=1)&(df02['duration']<=60)]

In [32]:
df02['PULocationID'] = df02['PULocationID'].astype(str)
df02['DOLocationID'] = df02['DOLocationID'].astype(str)

In [37]:
val_set = df02[['PULocationID', 'DOLocationID']].to_dict(orient = 'records')

In [40]:
X_val = dv.transform(val_set)
y_val = df02['duration'].values

In [42]:
y_val_pred = prediction_model.predict(X_val)

In [43]:
print('VAL RMSE: ', root_mean_squared_error(y_val, y_val_pred))

VAL RMSE:  7.811817745843695


### Ans: 7.81